In [45]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import random_split
from callbacks import EarlyStopping
import torchvision.transforms as transforms

np.random.seed(123)
torch.manual_seed(123)

In [46]:
# データの準備
path = "D:Statistics/data/deep_leraning/"
root = os.path.join(path, "torch", "mnist")
transform = transforms.Compose([transforms.ToTensor(), lambda x: x.view(-1)])
mnist_train = datasets.MNIST(root=root, download=True, train=True, transform=transform)
mnist_test = datasets.MNIST(root=root, download=True, train=False, transform=transform)

n_samples = len(mnist_train)
n_train = int(n_samples * 0.8)
n_val = n_samples - n_train
mnist_train, mnist_val = random_split(mnist_train, {n_train, n_val})

train_dataloader = DataLoader(mnist_train, batch_size=100, shuffle=True)
val_dataloader = DataLoader(mnist_val, batch_size=100, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size=100, shuffle=False)

In [47]:
# DNNを定義
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, rate):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.ReLU()
        self.d1 = nn.Dropout(rate)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.ReLU()
        self.d2 = nn.Dropout(rate)
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.ReLU()
        self.d3 = nn.Dropout(rate)
        self.l4 = nn.Linear(hidden_dim, output_dim)
        self.layers = [self.l1, self.a1, self.d1, self.l2, self.a2, self.d2, self.l3, self.a3, self.d3, self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [48]:
# モデルの学習
# アルゴリズムの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = np.unique(train_dataloader.dataset.dataset.classes).shape[0]
m = np.prod((train_dataloader.dataset.dataset.data).shape[1:])
rate = 0.5
model = DNN(m, 200, classes, rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.025)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

def val_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

# アルゴリズムの設定
epochs = 1000
es = EarlyStopping(patience=5, verbose=1)
hist = {"val_loss" : [], "val_accuracy": []}

In [52]:
# 確率的勾配法でモデルを学習
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0

    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    
    for (x, t) in val_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = val_step(x, t)
        val_loss += loss.item()
        val_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())
        
    val_loss /= len(val_dataloader)
    val_acc /= len(val_dataloader)
    
    hist["val_loss"].append(val_loss)
    hist["val_accuracy"].append(val_acc)
    print("epoch: {}, loss: {:.3}, acc: {:.3f}, val_loss: {:.3}, val_acc: {:.3f}".format(
        epoch+1, train_loss, train_acc, val_loss, val_acc))
    
    if es(val_loss):   # 早期終了判定
        break

epoch: 1, loss: 1.28, acc: 0.571, val_loss: 0.677, val_acc: 0.799
epoch: 2, loss: 0.752, acc: 0.760, val_loss: 0.437, val_acc: 0.876
epoch: 3, loss: 0.565, acc: 0.830, val_loss: 0.351, val_acc: 0.898
epoch: 4, loss: 0.473, acc: 0.861, val_loss: 0.294, val_acc: 0.915
epoch: 5, loss: 0.412, acc: 0.882, val_loss: 0.258, val_acc: 0.924
epoch: 6, loss: 0.37, acc: 0.896, val_loss: 0.231, val_acc: 0.933
epoch: 7, loss: 0.334, acc: 0.904, val_loss: 0.212, val_acc: 0.937
epoch: 8, loss: 0.31, acc: 0.912, val_loss: 0.197, val_acc: 0.943
epoch: 9, loss: 0.289, acc: 0.918, val_loss: 0.18, val_acc: 0.947
epoch: 10, loss: 0.269, acc: 0.924, val_loss: 0.17, val_acc: 0.950
epoch: 11, loss: 0.256, acc: 0.928, val_loss: 0.16, val_acc: 0.952
epoch: 12, loss: 0.24, acc: 0.932, val_loss: 0.156, val_acc: 0.953
epoch: 13, loss: 0.228, acc: 0.935, val_loss: 0.144, val_acc: 0.956
epoch: 14, loss: 0.221, acc: 0.938, val_loss: 0.141, val_acc: 0.958
epoch: 15, loss: 0.21, acc: 0.941, val_loss: 0.134, val_acc: 0.9